In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

animes_data_path = "./data/animes.csv"
profiles_data_path = "./data/profiles.csv"
reviews_data_path = "./data/reviews.csv"

In [ ]:
animes = pd.read_csv(animes_data_path)
profiles = pd.read_csv(profiles_data_path)
reviews = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)

<ipython-input-235-7f4491454ba1>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  reviews = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)


In [ ]:
# function to calculate sparsity given number of items, number of users, and number of ratings
def sparsity(num_items, num_users, num_ratings):
  return (num_items * num_users - num_ratings) / (num_users * num_items)

In [ ]:
print(animes.shape)
print(profiles.shape)
print(reviews.shape)

(19311, 12)
(81727, 5)
(192112, 7)


In [ ]:
# drop all duplicates by primary key
animes.drop_duplicates('uid', keep = 'first', inplace=True, ignore_index = True)
animes.reset_index(drop = True, inplace=True)

profiles.drop_duplicates('profile', keep = 'first', inplace=True, ignore_index = True)
profiles.reset_index(drop = True, inplace=True)

reviews.drop_duplicates(subset = ['profile', 'uid'], keep = 'first', inplace=True, ignore_index = True)
reviews.reset_index(drop = True, inplace=True)

In [ ]:
# shapes after dropping duplicates match exactly with what is given on the Kaggle dataset description
print(animes.shape)
print(profiles.shape)
print(reviews.shape)

(16216, 12)
(47885, 5)
(130519, 7)


In [ ]:
# drop age column initially for base model setup
profiles.drop("birthday", axis='columns', inplace=True)
profiles.reset_index(drop = True, inplace=True)

# drop animes which are only Hentai, and (optionally) drop animes with only one episode (feel free to change and not have it dropped)
animes = animes[animes.genre != "['Hentai']"]
animes = animes[animes.episodes != 1]
print(animes.shape)

# drop all review entries that do not have the animes or profiles after the above preprocessing steps
reviews = reviews[reviews.anime_uid.isin(animes.uid)]
reviews = reviews[reviews.profile.isin(profiles.profile)]
print(reviews.shape)

(7807, 12)
(110295, 7)


In [ ]:
# remove all reviews of users who have given lesser than three reviews
freq = reviews['profile'].value_counts()
frequent_values = freq[freq >= 3].index
reviews = reviews[reviews['profile'].isin(frequent_values)]
print(reviews.shape)

# Remove all users from profiles dataset who have given lesser than three reviews
profiles = profiles[profiles.profile.isin(reviews.profile)]
print(profiles.shape)

# remove all animes after the above two preprocessing steps
animes = animes[animes.uid.isin(reviews.anime_uid)]
print(animes.shape)

(69189, 7)
(9308, 4)
(4138, 12)


In [ ]:
# current sparsity of the dataset after preprocessing
print(sparsity(animes.shape[0], profiles.shape[0], reviews.shape[0]))

0.9982036531664452


In [ ]:
# TODO: Impute gender with some probability in profiles dataset
# TODO: Build user-item matrix after preprocessing is complete
# Note: The matrix can be made less sparse by using favorite_animes column to assign more scores (before CBCF)